# tensorflow

In [1]:
import tensorflow as tf
import numpy as np
from utils import (reset_tf_graph, show_tf_graph)



In [2]:
reset_tf_graph()
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
print(f)

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

print(sess.run(f))
sess.close()



Tensor("add_1:0", shape=(), dtype=int32)
42


In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    print(f.eval())

42


In [4]:
print(f.graph is tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    print(f.eval())

True
42


In [5]:
reset_tf_graph()

sess = tf.InteractiveSession()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
    

init = tf.global_variables_initializer()    
init.run()
result = f.eval()
print(result)

print(x.graph is tf.get_default_graph())

sess.close()

42
True


In [6]:
graph = tf.Graph() # 创建新的图

with graph.as_default(): # 在with块下作为默认图
    x2 = tf.Variable(2) # 定义的变量自动放到当前的默认图下

print(x2.graph is graph) # true
print(x2.graph is tf.get_default_graph()) # false

True
False


## tensorflow线性回归

In [7]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

m, n = housing.data.shape

print("housing data's shape: {}".format((m,n)))

print("housing target's shape: ", housing.target.shape)

# 添加偏置项x0 = 1
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print("housing data's shape with bias: ", housing_data_plus_bias.shape)

housing data's shape: (20640, 8)
housing target's shape:  (20640,)
housing data's shape with bias:  (20640, 9)


In [8]:
reset_tf_graph()

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")

y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y") 
# target的shape要转成(m, 1), reshape的第一个-1表示根据数量自动推算

XT = tf.transpose(X) # X的转置矩阵

# 使用最小二乘法计算theta
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    print(theta.eval())

[[-3.68901253e+01]
 [ 4.36643779e-01]
 [ 9.45042260e-03]
 [-1.07117996e-01]
 [ 6.43712580e-01]
 [-3.96291580e-06]
 [-3.78801115e-03]
 [-4.20931637e-01]
 [-4.34006572e-01]]


## tensorflow 批量梯度下降

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # 标准化
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [10]:
reset_tf_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") # 使用均匀分布初始化权重
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
print("best theta: ", best_theta)

Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.71450037
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.54363626
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
best theta:  [[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770885]
 [ 0.3617837 ]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.63752776]]


## MINI Batch

In [11]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()

housing_data_scaled = scaler.fit_transform(housing.data)
housing_data_scaled_bias = np.c_[np.ones((m, 1)), housing_data_scaled]

In [12]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "chpt9_tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

reset_tf_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

with tf.name_scope("loss") as scope:
    y_pred =  tf.matmul(X, theta, name="predictions")
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse") # error的平方再计算平均值既为均方差

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

mse_summay = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

show_tf_graph(tf.get_default_graph())

In [13]:


n_epochs=10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = housing_data_scaled_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summay.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

print("best theat in mini-batch: ", best_theta)

best theat in mini-batch:  [[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


## 模块化

In [17]:
reset_tf_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1) # get_shape返回的是TensorShape & Dimension类型, 需要转成int
        w = tf.Variable(tf.random_uniform(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0.0, name="max")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

# show_tf_graph(tf.get_default_graph())

file_writer = tf.summary.FileWriter("./chpt9_tf_logs/relus", tf.get_default_graph())

## 共享变量

In [19]:
reset_tf_graph()

def relu(X):
    with tf.name_scope("relu"):
        threshold = tf.get_variable("threhold", shape=(), initializer=tf.constant_initializer(0.0))
        
        w_shape = (int(X.get_shape()[1]), 1) # get_shape返回的是TensorShape & Dimension类型, 需要转成int
        w = tf.Variable(tf.random_uniform(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

relus = []
for i in range(5):
    with tf.variable_scope("relu", reuse=(i >= 1)) as scope:
        relus.append(relu(X))

output = tf.add_n(relus, name="output")

# show_tf_graph(tf.get_default_graph())

file_writer = tf.summary.FileWriter("./chpt9_tf_logs/relus_shared", tf.get_default_graph())